In [2]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QCoreApplication
import sys
import platform
if platform.system() == "Windows":
    import winsound
import cv2
import numpy as np

## 6.2 PyQt를 이용한 사용자 인터페이스

In [2]:
class BeepSound(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("삑 소리 내기")
        self.setGeometry(200, 200, 500, 100)
        
        short_beep_button = QPushButton("짧게 삑", self)
        long_beep_button = QPushButton("길게 삑", self)
        quit_button = QPushButton("나가기", self)
        
        self.label = QLabel("환영합니다!", self)
        
        short_beep_button.setGeometry(10, 10, 100, 30)
        long_beep_button.setGeometry(110, 10, 100, 30)
        quit_button.setGeometry(210, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 70)
        
        short_beep_button.clicked.connect(self.short_beep_function)
        long_beep_button.clicked.connect(self.long_beep_function)
        quit_button.clicked.connect(self.quit_function)
        
    def short_beep_function(self):
        self.label.setText("주파수 1000으로 0.5초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 500)
    
    def long_beep_function(self):
        self.label.setText("주파수 1000으로 3초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 3000)
        
    def quit_function(self):
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = BeepSound()
win.show()
app.exec_()

In [3]:
class Video(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("비디오에서 프레임 수집")
        self.setGeometry(200, 200, 500, 100)
        
        video_button = QPushButton("비디오 켜기", self)
        capture_button = QPushButton("프레임 잡기", self)
        save_button = QPushButton("프레임 저장", self)
        quit_button = QPushButton("나가기", self)
        
        video_button.setGeometry(10, 10, 100, 30)
        capture_button.setGeometry(110, 10, 100, 30)
        save_button.setGeometry(210, 10, 100, 30)
        quit_button.setGeometry(310, 10, 100, 30)
        
        video_button.clicked.connect(self.video_function)
        capture_button.clicked.connect(self.capture_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
    def video_function(self):
        self.cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not self.cap.isOpened(): self.close()
        
        while True:
            ret, self.frame = self.cap.read()
            if not ret: break
            cv2.imshow("Video display", self.frame)
            cv2.waitKey(1)
        
    def capture_function(self):
        self.capture_frame = self.frame
        cv2.imshow("Captured Frame", self.frame)
    
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", "./")
        cv2.imwrite(f_name[0], self.capture_frame)
        
    def quit_function(self):
        self.cap.release()
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Video()
win.show()
app.exec_()

0

## 6.3 오림

In [2]:
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("오림")
        self.setGeometry(200, 200, 700, 200)
        
        file_button = QPushButton("파일", self)
        paint_button = QPushButton("페인팅", self)
        cut_button = QPushButton("오림", self)
        inc_button = QPushButton('+', self)
        dec_button = QPushButton('-', self)
        save_button = QPushButton("저장", self)
        quit_button = QPushButton("나가기", self)
        
        
        file_button.setGeometry(10, 10, 100, 30)
        paint_button.setGeometry(110, 10, 100, 30)
        cut_button.setGeometry(210, 10, 100, 30)
        inc_button.setGeometry(310, 10, 50, 30)
        dec_button.setGeometry(360, 10, 50, 30)
        save_button.setGeometry(410, 10, 100, 30)
        quit_button.setGeometry(510, 10, 100, 30)
        
        file_button.clicked.connect(self.file_open_function)
        paint_button.clicked.connect(self.painting_function)
        cut_button.clicked.connect(self.cut_function)
        inc_button.clicked.connect(self.inc_function)
        dec_button.clicked.connect(self.dec_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
        self.brush_size = 5
        self.left_color, self.right_color = (255, 0, 0), (0, 0, 255)
        
    def file_open_function(self):
        f_name = QFileDialog.getOpenFileName(self, "open file", "./")
        self.img = cv2.imread(f_name[0])
        if self.img is None: sys.exit("파일을 찾을 수 없습니다.")
        
        self.img_show = np.copy(self.img)
        cv2.imshow("Painting", self.img_show)
        
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv2.GC_PR_BGD
        
    def painting_function(self):
        cv2.setMouseCallback("Painting", self.painting)
        
    def painting(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_RBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        
        cv2.imshow("Painting", self.img_show)
        
    def cut_function(self):
        background = np.zeros((1, 65), np.float64)
        foreground = np.zeros((1, 65), np.float64)
        
        cv2.grabCut(self.img, self.mask, None, background, foreground, 5, cv2.GC_INIT_WITH_MASK)
        mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype("uint8")
        self.grabImg = self.img*mask2[..., np.newaxis]
        cv2.imshow("Scissoring", self.grabImg)
        
    def inc_function(self):
        self.brush_size = min(20, self.brush_size + 1)
        
    def dec_function(self):
        self.brush_size = max(1, self.brush_size - 1)
        
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", './')
        cv2.imwrite(f_name[0], self.grabImg)
        
    def quit_function(self):
        cv2.destroyAllWindows()
        self.close()

        
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

0